In [1]:
# # this block of code imports graphical libraries for plotting graphs with high resolution
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120

In [54]:
# Libraries of functions need to be imported
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy import linalg
from scipy.interpolate import interp1d, LinearNDInterpolator, NearestNDInterpolator
from sklearn.decomposition import PCA

from scipy.spatial.distance import cdist

# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

# Lowess with distance calculations and boosting

In [3]:
# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))

In [4]:
# this is the correct vectorized version
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)

In [5]:
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))

In [6]:
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

# Learning Objectives

-  How to efficiently compute all the pairwise distances among different observations.
- How to use the weights with the distances.
- Test the locally weighted regressors.
- Develop applications with boosting techniques.

In [7]:
# real data application includes cars.csv, concrete.csv and more
data = pd.read_csv('../Data Sets/cars.csv')

In [8]:
data

,MPG,CYL,ENG,WGT
0,18.0,8,307.0,3504
1,15.0,8,350.0,3693
2,18.0,8,318.0,3436
3,16.0,8,304.0,3433
4,17.0,8,302.0,3449
...,...,...,...,...
387,27.0,4,140.0,2790
388,44.0,4,97.0,2130
389,32.0,4,135.0,2295
390,28.0,4,120.0,2625


In [8]:
x = data.drop(columns=['MPG']).values
y = data['MPG'].values.reshape(-1,1)

## Compute all pairwise distances efficiently

This means we build a general case, assuming two matrices, we want to compute all the pairwise distances between the rows of matrix 1 and the rows of matrix 2.

The Euclidean distance between $\vec{u}$ and $\vec{v}$ is:

$$\large \text{dist}\left(\vec{u},\vec{v}\right):= \sqrt{\sum(u_i-v_i)^2}$$

In [ ]:
%time
cdist(x, x[0:2], metric='euclidean')

A better choice would be to use Usearch.

# Important

In [13]:
def weight_function(u,v,kern=Gaussian,tau=0.5):
    return kern(cdist(u, v, metric='euclidean')/(2*tau))

## Scaling is very important for Locally Weighted Regression

In [10]:
# we show now the effect of scaling onto the data
scale = MinMaxScaler()

In [ ]:
# Example of the weights matrix if we used all the data
# provided that you have defined xtrain and xtest
# W = weight_function(scale.fit_transform(xtrain),scale.transform(xtest),kern=Tricubic,tau=0.3)

# The Lowess Class - Version without Triangulation

In [47]:
class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.0001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = []
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test.append(lm.predict([x_new[i]]))
        return np.array(yest_test).flatten()

# We benchmark on a complete k-Fold cross-validation

In [14]:
mse_lwr = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_rf = RandomForestRegressor(n_estimators=300,max_depth=12)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_lw = Lowess(kernel= Epanechnikov,tau=0.14)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 16.455751824171227
The Cross-validated Mean Squared Error for Random Forest is : 18.41391555214139


In [37]:
scale = MinMaxScaler()

# Boosting Applications

We implement Gradient Boosting

In plain words:

 - you fit the model1 on the train
 - you obtain the residuals from predicting the ytrain
 - you fit another model on the xtrain and the residuals of model1: i.e. model2.fit(xtrain,residuals_train)
 - you predict the xtest by using (model1 + model2).predict(xtest)

## Work with Different Implementations of Locally Weighted Regression

In [20]:
def lw_ag_md(x, y, xnew,f=2/3,iter=3, intercept=True):

  n = len(x)
  r = int(np.ceil(f * n))
  yest = np.zeros(n)

  if len(y.shape)==1: # here we make column vectors
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  def dist(u,v):
    if len(v.shape)==1:
      v = v.reshape(1,-1)
      d = np.sqrt(np.sum((u-v)**2,axis=1))
    else:
      d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
    return d

  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]
  # dist(x,x) is always symmetric
  w = np.clip(dist(x,x) / h, 0.0, 1.0)
  w = (1 - w ** 3) ** 3

  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(delta).dot(np.diag(w[i,:]))
      # when we multiply two diagional matrices we get also a diagonal matrix
      b = np.transpose(x1).dot(W).dot(y)
      A = np.transpose(x1).dot(W).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1]) # if we want L2 regularization
      beta = linalg.solve(A, b)
      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta)[0]

    residuals = y.ravel() - yest
    s = np.median(np.abs(residuals))

    delta = np.clip(residuals / (6.0 * s), -1, 1)

    delta = (1 - delta ** 2) ** 2

  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ')
      f = LinearNDInterpolator(tri,yest[ind])
      output[i] = f(pca.transform(xnew[i].reshape(1,-1))).flatten()[0] # the output may have NaN's where the data points from xnew are outside the convex hull of X
  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,y.ravel())
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)]).flatten()[0]
  return output

In [21]:
class Lowess_AG_MD:
    def __init__(self, f = 1/10, iter = 3,intercept=True):
        self.f = f
        self.iter = iter
        self.intercept = intercept

    def fit(self, x, y):
        f = self.f
        iter = self.iter
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        f = self.f
        iter = self.iter
        intercept = self.intercept
        return lw_ag_md(x, y, x_new, f, iter, intercept) # this is actually our defined function of Lowess

    def get_params(self, deep=True):
    # suppose this estimator has parameters "f", "iter" and "intercept"
        return {"f": self.f, "iter": self.iter,"intercept":self.intercept}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

## We want a fair comparison

In [22]:
mse_lwr = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=4)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel= Epanechnikov,tau=0.14)
model_2 = Lowess(kernel= Gaussian,tau=0.2)
#model_2 = Lowess_AG_MD(f=1/3,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 68.56443378608267
The Cross-validated Mean Squared Error for a DT-based method: 21.92482466336871


## Applications with Different Higher Dimensional Data Sets

In [8]:
data = pd.read_csv('../Data Sets/concrete.csv')

In [21]:
data

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [9]:
x = data.drop(columns=['strength']).values
y = data['strength'].values

In [27]:
y

array([79.99, 61.89, 40.27, ..., 23.7 , 32.77, 32.4 ])

In [58]:
mse_lwr = []
mse_rf = []
scale = QuantileTransformer(n_quantiles=900)
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel=Gaussian,tau=0.4)
model_2 = Lowess(kernel=Tricubic,tau=0.3)
#model_1 = Lowess_AG_MD(f=1/4,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 20.944520624684298
The Cross-validated Mean Squared Error for a DT-based method: 20.18308623182933
